<a href="https://colab.research.google.com/github/mazidzomader/CSE422-PYTHON_BRACU/blob/main/Lab%20Materials/Lab%2002%20-%20Genetic%20Algorithm/CSE422_Lab_02%5BClass%20Version%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 01

In [10]:
import random
def chromosomeEncoding(grid, comp, popu):
    dict = {}
    for i in range(popu):
        dict[f"P{i}"] = []
        for ii in range(len(comp)):
            width = comp[ii][1]
            height = comp[ii][2]
            temp_w = random.randint(0, grid-width)
            temp_h = random.randint(0, grid-height)
            dict[f"P{i}"].append((temp_w, temp_h))

    return dict



In [11]:
def fitnessFnDesign(chr, comps):
    #Measuring Coordinates
    directions = {}
    for i in range(len(comps)):
        coord = chr[i]
        x = coord[0]
        y = coord[1]

        w = comps[i][1]
        h = comps[i][2]

        left = x
        right = x+w
        bottom = y
        top = y+h
        center = (x+w/2, y +h/2)
        directions[f"p{i}"] = [left, right, bottom, top, center]


    # Overlap checkings
    ovl_cnt = 0
    for i in range(len(comps)):
        A = directions[f"p{i}"]
        for j in range(i+1, len(comps)):
            B = directions[f"p{j}"]

            if not (A[1] <= B[0] or
                    A[0] >= B[1] or
                    A[3] <= B[2] or
                    A[2] >= B[3]):
                ovl_cnt += 1

    # total wiring length
    totalWire = 0
    for i in range(len(comps)):
        A = directions[f"p{i}"]
        for j in range(i+1, len(comps)):
            B = directions[f"p{j}"]

            dx = A[4][0] - B[4][0]
            dy = A[4][1] - B[4][1]

            totalWire += (dx**2+dy**2)**0.5

    # bounding box area
    x_min = min([directions[f"p{i}"][0] for i in range(len(comps))])
    y_min = min([directions[f"p{i}"][2] for i in range(len(comps))])
    x_max = max([directions[f"p{i}"][1] for i in range(len(comps))])
    y_max = max([directions[f"p{i}"][3] for i in range(len(comps))])

    Area = (x_max-x_min) * (y_max-y_min)


    # fitness = -(α * overlaps + β * wiring + γ * area)

    α, β, γ= 1000, 2, 1

    fitness = -(α * ovl_cnt + β*totalWire + γ* Area)

    return fitness


In [12]:
def parentSelection(chr_fit):
    sorted_list = sorted(chr_fit, key=lambda x: x[1], reverse=True)

    parent1 = sorted_list[0]
    parent2 = sorted_list[1]

    return parent1, parent2

In [13]:
def crossOver(p1, p2):
    assert len(p1) == len(p2)
    # Using Two point Crossover
    idx1 = 2
    idx2 = 4

    c1 = p1[:idx1] + p2[idx1:idx2] + p1[idx2:]
    c2 = p2[:idx1] + p1[idx1:idx2] + p2[idx2:]


    return c1, c2

In [14]:
def mutation(chr, comp, grid, mutate = 0.1):
    for i in range(len(chr)):
        if random.random() < mutate:
            w, h = comp[i][1], comp[i][2]
            x_new = random.randint(0, grid-w)
            y_new = random.randint(0, grid-h)
            chr[i] = (x_new, y_new)
    return chr

In [15]:
# === DRIVER CODE ===
grid_size = 25
components = [
    ("ALU", 5, 5),
    ("Cache", 7, 4),
    ("Control", 4, 4),
    ("Register", 6, 6),
    ("Decoder", 5, 3),
    ("Floating", 5, 5)
]
population_size = 6
num_generations = 15  # evolve for 15 generations

# Initialize population
enc = chromosomeEncoding(grid_size, components, population_size)

# Evolution process
for gen in range(num_generations):
    # Step 1 — Evaluate fitness
    fitness_list = []
    for i in enc.keys():
        fitness = fitnessFnDesign(enc[i], components)
        fitness_list.append((i, fitness))

    # Step 2 — Select parents
    p1, p2 = parentSelection(fitness_list)

    # Step 3 — Crossover
    c1, c2 = crossOver(enc[p1[0]], enc[p2[0]])

    # Step 4 — Mutation
    c1 = mutation(c1, components, grid_size)
    c2 = mutation(c2, components, grid_size)

    # Step 5 — Replace worst chromosomes (simple version)
    # Sort by fitness (lowest = worst)
    sorted_fit = sorted(fitness_list, key=lambda x: x[1])
    worst1 = sorted_fit[0][0]
    worst2 = sorted_fit[1][0]

    enc[worst1] = c1
    enc[worst2] = c2

    # Step 6 — Track progress
    best = max(fitness_list, key=lambda x: x[1])
    print(f"Generation {gen+1}: Best Fitness = {best[1]:.2f}")

# 3️⃣ Final Output
print("\nBest Layout Found:")
print("Chromosome:", best[0])
print("Fitness:", best[1])
print("Coordinates:", enc[best[0]])


Generation 1: Best Fitness = -841.10
Generation 2: Best Fitness = -841.10
Generation 3: Best Fitness = -841.10
Generation 4: Best Fitness = -841.10
Generation 5: Best Fitness = -841.10
Generation 6: Best Fitness = -841.10
Generation 7: Best Fitness = -841.10
Generation 8: Best Fitness = -841.10
Generation 9: Best Fitness = -841.10
Generation 10: Best Fitness = -841.10
Generation 11: Best Fitness = -841.10
Generation 12: Best Fitness = -841.10
Generation 13: Best Fitness = -841.10
Generation 14: Best Fitness = -841.10
Generation 15: Best Fitness = -841.10

Best Layout Found:
Chromosome: P2
Fitness: -841.1002021109464
Coordinates: [(9, 12), (5, 5), (10, 0), (6, 19), (1, 11), (16, 6)]


# Task 02

In [16]:
print(enc)
parents = random.sample(list(enc.keys()), 2)
p1, p2 = parents[0], parents[1]

idx1 = random.randint(0, len(components) - 2)
idx2 = random.randint(idx1 + 1, len(components) - 1)

print(f"Selected Parents: {p1}, {p2}")
print(f"Crossover Points: {idx1}, {idx2}")

c1 = enc[p1][:idx1] + enc[p2][idx1:idx2] + enc[p1][idx2:]
c2 = enc[p2][:idx1] + enc[p1][idx1:idx2] + enc[p2][idx2:]

print("Parent 1:", enc[p1])
print("Parent 2:", enc[p2])


print("Child 1:", c1)
print("Child 2:", c2)


{'P0': [(12, 14), (14, 2), (18, 11), (8, 2), (2, 0), (14, 19)], 'P1': [(12, 14), (14, 2), (10, 0), (1, 6), (2, 0), (14, 19)], 'P2': [(9, 12), (5, 5), (10, 0), (6, 19), (1, 11), (16, 6)], 'P3': [(9, 12), (5, 5), (18, 11), (8, 2), (1, 11), (16, 6)], 'P4': [(12, 14), (14, 2), (10, 0), (6, 19), (10, 4), (12, 13)], 'P5': [(12, 14), (14, 2), (10, 0), (6, 19), (2, 0), (14, 19)]}
Selected Parents: P0, P5
Crossover Points: 2, 3
Parent 1: [(12, 14), (14, 2), (18, 11), (8, 2), (2, 0), (14, 19)]
Parent 2: [(12, 14), (14, 2), (10, 0), (6, 19), (2, 0), (14, 19)]
Child 1: [(12, 14), (14, 2), (10, 0), (8, 2), (2, 0), (14, 19)]
Child 2: [(12, 14), (14, 2), (18, 11), (6, 19), (2, 0), (14, 19)]
